In [40]:
import importlib
import copy
import pickle
# --------------------
from sklearn.utils import gen_batches, check_array
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.metrics import root_mean_squared_error as RMSE
from sklearn.utils.validation import FLOAT_DTYPES
import numpy as np
from numpy.linalg import norm,inv,matrix_rank, pinv
# --------------------
from skopt.space import Integer
from skopt.utils import use_named_args
from skopt import gp_minimize
# --------------------
import h5py
from sklearn.cross_decomposition import PLSRegression

# import warnings
# warnings.filterwarnings('ignore')

from pytictoc import TicToc
tim=TicToc()
tim_tot = TicToc()

In [41]:
def RollingCV(tscv,X):
    cv = tscv.split(X)
    (train_index, test_index) = next(cv)
    yield(
        train_index,
        test_index
    )
    test_size = len(test_index)

    for (train_index, test_index) in (cv):
        yield(
            train_index[-test_size:],
            test_index
        )

In [ ]:
with h5py.File('./data/merra2_t.h5', 'r') as f:
    X_train, Y_train = f['X_train'], f['Y_train']
    n_train = X_train.shape[0]
    print(n_train)

    X_train = X_train[0:n_train]
    Y_train = Y_train[0:n_train]

n_fold = 74
test_size=30    

In [42]:
with h5py.File('./data/TW_PM25.h5', 'r') as f:
    X_train, Y_train = f['X_train'], f['Y_train']
    n_train = X_train.shape[0]
    print(n_train)
        
    X_train = X_train[0:n_train]
    Y_train = Y_train[0:n_train]
n_fold = 17
test_size=30

877


In [60]:
def dump_PLS(PLS, fn):
    np.savez(fn,
             n=PLS.n,
             n_comp=PLS.n_components,
             W = PLS.W,
             y_loadings = np.dot(PLS.S.T, PLS.W),
             CW = np.dot(PLS.C, PLS.W),
             x_mean = PLS._x_mean,
             y_mean = PLS._y_mean
             )

def predict(dat, X, n_comp):
    X = check_array(X, copy=copy, dtype=FLOAT_DTYPES)
    W = dat['W'][:,:n_comp]
    CW = dat['CW'][:,:n_comp]
    y_loadings = dat['y_loadings'][:,:n_comp]

    x_rotaions = np.dot(
        W,
        pinv(np.dot(W.T,
                    CW))
    )

    coef = np.dot(x_rotaions, y_loadings.T)
    X -= dat['x_mean']
    ypred = np.dot(X, coef)
    ypred += dat['y_mean']

    return ypred


In [53]:
print(X_train.shape)
print(Y_train.shape)

tscv = TimeSeriesSplit(n_splits=n_fold, test_size=test_size)
for i, (train_index, test_index) in enumerate(tscv.split(X_train)):
    print(f"Fold {i}:")
    print(f"  Train: len={len(train_index)}")
    print(f"  Test:  len={len(test_index)}")
    if(i>3):
        print(f"  Train: {train_index}")
        print(f"  Test:  {test_index}")
        break

print("="*40)
for i, (train_index, test_index) in enumerate(RollingCV(tscv,X_train)):
    print(f"Fold {i}:")
    print(f"  Train: len={len(train_index)}")
    print(f"  Test:  len={len(test_index)}")
    if(i>3):
        print(f"  Train: {train_index}")
        print(f"  Test:  {test_index}")
        break

(877, 1800)
(877, 1800)
Fold 0:
  Train: len=367
  Test:  len=30
Fold 1:
  Train: len=397
  Test:  len=30
Fold 2:
  Train: len=427
  Test:  len=30
Fold 3:
  Train: len=457
  Test:  len=30
Fold 4:
  Train: len=487
  Test:  len=30
  Train: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187

In [54]:
import Code.ISIMPLS
importlib.reload(Code.ISIMPLS)
from Code.ISIMPLS import ISIMPLS2 as ISIMPLS

n_comp_list=list(range(1,21))
print("n_comp:", n_comp_list)
score_all = np.zeros((n_fold, len(n_comp_list)))

n_comp_max=20
PLS = ISIMPLS(n_components=n_comp_max)

tim_tot.tic()
tim.tic()
for i,(train_index, test_index) in enumerate(RollingCV(tscv, X_train)):
    PLS.fit(X_train[train_index], Y_train[train_index])
    y_true = Y_train[test_index]
    for j,n_comp in enumerate(n_comp_list):
        y_pred = PLS.predict(X_train[test_index], n_comp)
        score_all[i,j] = RMSE(y_true, y_pred)
    if (i%5==0): 
        print(f"fold: {i+1:02d}, elapsed time: {tim.tocvalue():.1f}s")
        tim.tic()

scores = score_all.mean(axis=0)
best_ind = np.nanargmin(scores)
print("")
print(f"best parameter: {n_comp_list[best_ind]}; score: {scores[best_ind]:.7e}; "
      + f"total time={tim_tot.tocvalue():.1f}s")
print(scores)

n_comp: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]


fold: 01, elapsed time: 3.0s


fold: 06, elapsed time: 13.0s


fold: 11, elapsed time: 12.8s


fold: 16, elapsed time: 12.0s



best parameter: 11; score: 7.1476354e+00; total time=43.2s
[7.76705994 7.50395476 7.36823013 7.31293782 7.31652096 7.29054187
 7.2979135  7.2500481  7.23974823 7.16567089 7.14763537 7.19020176
 7.19894527 7.21953187 7.24954174 7.27576464 7.32228498 7.33506288
 7.33961495 7.38431746]


In [81]:
import Code.ISIMPLS
importlib.reload(Code.ISIMPLS)
from Code.ISIMPLS import ISIMPLS2 as ISIMPLS

n_comp_max=100
PLS = ISIMPLS(n_components=n_comp_max)
tim_tot.tic()
tim.tic()
for i,(train_index, test_index) in enumerate(RollingCV(tscv, X_train)):
    PLS.fit(X_train[train_index], Y_train[train_index])
    dump_PLS(PLS, f'./data/wrk/ISIMPLS_fold{i:02d}.npz')
    if (i%5==0):
        print(f"fold: {i+1:02d}, elapsed time: {tim.tocvalue():.1f}s")
        tim.tic()
print(f"total time={tim_tot.tocvalue():.1f}s")

fold: 01, elapsed time: 16.9s


fold: 06, elapsed time: 78.0s


fold: 11, elapsed time: 76.4s


fold: 16, elapsed time: 79.3s


total time=266.3s


In [82]:
PLS_list = []
for i in range(n_fold):
    PLS_list.append(np.load(f'./data/wrk/ISIMPLS_fold{i:02d}.npz'))


In [83]:
space  = [Integer(1, n_comp_max, name='n_components')]
@use_named_args(space)
def Comp_Model_Score(n_components):
    scores = np.zeros((n_fold,))
    for i,(train_index, test_index) in enumerate(RollingCV(tscv, X_train)):
        y_true = Y_train[test_index]
        y_pred = predict(PLS_list[i], X_train[test_index], n_components)
        # y_pred = predict(np.load(f'./data/wrk/ISIMPLS_fold{i:02d}.npz'),
        #                  X_train[test_index], n_components)
        scores[i] = RMSE(y_true, y_pred)
    return np.mean(scores)


In [84]:
n_calls = int(np.log(n_comp_max)) + 1
print(f"n_calls = {n_calls:d}")
n_calls *= 2

tim_tot.tic()
res_gp = gp_minimize(Comp_Model_Score, space, n_calls=max(n_calls,10), 
                     random_state=0, verbose=True)

print("-"*40)
print(f"Best param = {res_gp.x[0]:02d}")
print(f"Best score = {res_gp.fun:.4f}")
print(f"total time = {tim_tot.tocvalue():.1f}s")


n_calls = 5
Iteration No: 1 started. Evaluating function at random point.


Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.6960
Function value obtained: 8.1527
Current minimum: 8.1527
Iteration No: 2 started. Evaluating function at random point.


Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.7123
Function value obtained: 8.3838
Current minimum: 8.1527
Iteration No: 3 started. Evaluating function at random point.


Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.7060
Function value obtained: 8.3889
Current minimum: 8.1527
Iteration No: 4 started. Evaluating function at random point.


Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.7026
Function value obtained: 8.3838
Current minimum: 8.1527
Iteration No: 5 started. Evaluating function at random point.


...

Iteration No: 6 ended. Evaluation done at random point.
Time taken: 0.6009
Function value obtained: 7.8384
Current minimum: 7.8384
Iteration No: 7 started. Evaluating function at random point.


Iteration No: 7 ended. Evaluation done at random point.
Time taken: 0.5468
Function value obtained: 7.6128
Current minimum: 7.6128
Iteration No: 8 started. Evaluating function at random point.


Iteration No: 8 ended. Evaluation done at random point.
Time taken: 0.5231
Function value obtained: 7.2979
Current minimum: 7.2979
Iteration No: 9 started. Evaluating function at random point.


Iteration No: 9 ended. Evaluation done at random point.
Time taken: 0.5547
Function value obtained: 7.5730
Current minimum: 7.2979
Iteration No: 10 started. Evaluating function at random point.


Iteration No: 10 ended. Evaluation done at random point.
Time taken: 0.9994
Function value obtained: 7.9823
Current minimum: 7.2979
----------------------------------------
Best param = 07
Best score = 7.2979
total time = 6.7s


In [ ]:
import Code.ISIMPLS
importlib.reload(Code.ISIMPLS)
from Code.ISIMPLS import ISIMPLS2 as ISIMPLS

scores = []
params = []
n_comp_list=list(range(100,401,50))

tim_tot.tic()
for n_comp in n_comp_list:
    tim.tic()
    pls=ISIMPLS(n_components=n_comp)
    scores.append(Comp_Model_Score(pls, RollingCV(tscv,X_train), X_train, Y_train) )
    params.append({'n_components': n_comp})
    print(f"params={params[-1]}, score={scores[-1]:.7e}, "
          +f"elapsed time={tim.tocvalue():.1f}s")

best_ind = np.nanargmin(scores)
print("")
print(f"best parameter: {params[best_ind]}; score: {scores[best_ind]:.7e}; "
      + f"total time={tim_tot.tocvalue():.1f}s")


params={'n_components': 100}, score=2.1730468e+00, elapsed time=97598.1s


params={'n_components': 150}, score=2.2380231e+00, elapsed time=159673.6s
